# AequilibraE Routing

Inputs:  demand, network

Outputs:  shortest path skims, routing results

## Major steps
1. Set up Aequilibrae environment
2. Obtain the shortest path skim from the network
3. Run routing 
4. Generate summary statistics


## Aequilibrae environment

In [1]:
#needs scipy, openmatrix (pip install)
import sys
from os.path import join
import numpy as np
import pandas as pd
import openmatrix as omx
from math import log10, floor
import matplotlib.pyplot as plt
from aequilibrae.distribution import GravityCalibration, Ipf, GravityApplication, SyntheticGravityModel
from aequilibrae import Parameters
from aequilibrae.project import Project
from aequilibrae.paths import PathResults
from aequilibrae.paths import SkimResults #as skmr
from aequilibrae.paths import Graph
from aequilibrae.paths import NetworkSkimming
from aequilibrae.matrix import AequilibraeData, AequilibraeMatrix
from aequilibrae import logger
from aequilibrae.paths import TrafficAssignment, TrafficClass
import logging

In [2]:
fldr = 'C:/Users/Scott.Smith/GMNS/Lima'  #was aeqRepro
proj_name = 'Lima.sqlite'   #the network comes from this sqlite database
dt_fldr = '0_tntp_data'
prj_fldr = '1_project'
skm_fldr = '2_skim_results'
assg_fldr = '4_assignment_results'

In [3]:
p = Parameters()
p.parameters['system']['logging_directory'] = fldr
p.write_back()

In [4]:
# Because assignment takes a long time, we want the log to be shown here
stdout_handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("%(asctime)s;%(name)s;%(levelname)s ; %(message)s")
stdout_handler.setFormatter(formatter)
logger.addHandler(stdout_handler)

## Shortest path skim

In [5]:
project = Project()
project.load(join(fldr, prj_fldr, proj_name))


2021-02-20 10:31:02,382;aequilibrae;WARNING ; AequilibraE might not work as intended without spatialite. ('The specified module could not be found.\r\n',)


In [6]:
# we build all graphs
project.network.build_graphs()
# We get warnings that several fields in the project are filled with NaNs.  Which is true, but we won't
# use those fields

# we grab the graph for cars
graph = project.network.graphs['c']

# let's say we want to minimize free_flow_time #distance
graph.set_graph('free_flow_time')

# And will skim time and distance while we are at it
graph.set_skimming(['free_flow_time', 'distance'])

# And we will allow paths to be compute going through other centroids/centroid connectors
# required for the Sioux Falls network, as all nodes are centroids
graph.set_blocked_centroid_flows(True)


C:\Users\Scott.Smith\AppData\Local\Continuum\anaconda3\envs\RDRenv\lib\site-packages\aequilibrae\project\network\network.py:279: UserWarning: Fields were removed form Graph for being non-numeric: modes,link_type
  warn(f'Fields were removed form Graph for being non-numeric: {",".join(removed_fields)}')


In [7]:
########## SKIMMING ###################

# setup the object result
res = SkimResults()
res.prepare(graph)

# And run the skimming
res.compute_skims()

# The result is an AequilibraEMatrix object
skims = res.skims

# We can export to OMX
skims.export(join(fldr, skm_fldr, 'sp_skim.omx'))   #change for each run  

## Routing

In [8]:
#### Open the matrix to get its size ####
f_demand = omx.open_file(join(fldr, dt_fldr, 'demand.omx'))
matrix_shape = f_demand.shape()
matrix_size = matrix_shape[1]
print('Base Skim Shape:',f_demand.shape(),  "Size=",matrix_size)
print('Number of tables',len(f_demand))
print('Table names:',f_demand.list_matrices())
print('attributes:',f_demand.list_all_attributes())

f_demand.close()

Base Skim Shape: (449, 449) Size= 449
Number of tables 1
Table names: ['matrix']
attributes: []


In [9]:
#### LOAD DEMAND MATRIX #####
demand = AequilibraeMatrix()
demand.load(join(fldr, dt_fldr, 'demand.omx'))
demand.computational_view(['matrix'])  # We will only assign one user class stored as 'matrix' inside the OMX file


In [10]:
######### TRAFFIC ASSIGNMENT WITH SKIMMING


assig = TrafficAssignment()

# Creates the assignment class
assigclass = TrafficClass(graph, demand)

# The first thing to do is to add at list of traffic classes to be assigned
assig.set_classes([assigclass])

assig.set_vdf("BPR")  # This is not case-sensitive # Then we set the volume delay function

assig.set_vdf_parameters({"alpha": "b", "beta": "power"})  # Get parameters from link file
#assig.set_vdf_parameters({"alpha": 0.15, "beta": 4}) 

assig.set_capacity_field("capacity")  # The capacity and free flow travel times as they exist in the graph
assig.set_time_field("free_flow_time")

# And the algorithm we want to use to assign
assig.set_algorithm('bfw')
#assig.set_algorithm('msa')   #all-or-nothing

# since I haven't checked the parameters file, let's make sure convergence criteria is good
assig.max_iter = 100      #was 1000  or 100
assig.rgap_target = 0.001   #was 0.00001,  or 0.01

assig.execute()  # we then execute the assignment

# The link flows are easy to export.
# we do so for csv and AequilibraEData
assigclass.results.save_to_disk(join(fldr, assg_fldr, 'linkflow.csv'), output="loads") #change for each run
#assigclass.results.save_to_disk(join(fldr, assg_fldr, 'link_flows_c1.aed'), output="loads")

# the skims are easy to get.

# The blended one are here
avg_skims = assigclass.results.skims

# The ones for the last iteration are here
last_skims = assigclass._aon_results.skims

# Assembling a single final skim file can be done like this
# We will want only the time for the last iteration and the distance averaged out for all iterations  
kwargs = {'file_name': join(fldr, assg_fldr, 'rt_skim'+'.aem'),   #change
          'zones': graph.num_zones,
          'matrix_names': ['time_final', 'distance_blended']}

# Create the matrix file
out_skims = AequilibraeMatrix()
out_skims.create_empty(**kwargs)
out_skims.index[:] = avg_skims.index[:]

# Transfer the data
# The names of the skims are the name of the fields
out_skims.matrix['time_final'][:, :] = last_skims.matrix['free_flow_time'][:, :]
# It is CRITICAL to assign the matrix values using the [:,:]
out_skims.matrix['distance_blended'][:, :] = avg_skims.matrix['distance'][:, :]

out_skims.matrices.flush()  # Make sure that all data went to the disk

# Export to OMX as well
out_skims.export(join(fldr, assg_fldr, 'rt_skim'+'.omx'))
demand.close()

2021-02-20 10:31:13,002;aequilibrae;INFO ; bfw Assignment STATS
2021-02-20 10:31:13,002;aequilibrae;INFO ; Iteration, RelativeGap, stepsize
2021-02-20 10:31:13,365;aequilibrae;INFO ; 1,inf,1.0
2021-02-20 10:31:13,735;aequilibrae;INFO ; 2,5.366701981010726e-07,0.9079656402038635
2021-02-20 10:31:14,089;aequilibrae;INFO ; 3,2.0979137149331117e-06,0.4622512458414412
2021-02-20 10:31:14,089;aequilibrae;INFO ; bfw Assignment finished. 3 iterations and 2.0979137149331117e-06 final gap


## Calculate summary statistics


In [11]:
f = omx.open_file(join(fldr, dt_fldr, 'demand.omx'),'r')  #change
print('DEMAND FILE Shape:',f.shape(),'   Tables:',f.list_matrices(),'   Mappings:',f.list_mappings())
dem = f['matrix']

spbf = omx.open_file(join(fldr, skm_fldr,'sp_skim.omx'),'r')    #change
print('SP BASE SKIM FILE Shape:',spbf.shape(),'   Tables:',spbf.list_matrices(),'   Mappings:',spbf.list_mappings())
spbt = spbf['free_flow_time']
spbd = spbf['distance']

rtbf = omx.open_file(join(fldr, assg_fldr, 'rt_skim.omx'),'r')
print('RT BASE SKIM FILE Shape:',rtbf.shape(),'   Tables:',rtbf.list_matrices(),'   Mappings:',rtbf.list_mappings())

rtbt = rtbf['time_final']
rtbd = rtbf['distance_blended']

DEMAND FILE Shape: (449, 449)    Tables: ['matrix']    Mappings: ['taz']
SP BASE SKIM FILE Shape: (449, 449)    Tables: ['distance', 'free_flow_time']    Mappings: ['main_index']
RT BASE SKIM FILE Shape: (449, 449)    Tables: ['distance_blended', 'time_final']    Mappings: ['main_index']


In [12]:
#Summary information on the input trip tables
print('sum of demand trips','{:.9}'.format(np.sum(dem)))

sum of demand trips 32041.0


### Skims as .csv files

In [13]:
outfile = open("combined_skim.txt","w")  #change
spb_cumtripcount = 0.0;
spb_cumtime = 0.0;
spb_cumdist = 0.0;
rtb_cumtime = 0.0;
rtb_cumdist = 0.0;
largeval = 999999;

#Shortest path base times and distances
print("i j demand sp_dist rt_dist sp_time rt_time",file=outfile)
for i in range(matrix_size):
    tripcount = 0.0;
    sp_timecount = 0.0;
    sp_distcount = 0.0;
    rt_timecount = 0.0;
    rt_distcount = 0.0;
    for j in range(matrix_size):
        if(dem[i][j]>0):
            tripcount = tripcount + dem[i][j]
            sp_timecount = sp_timecount + dem[i][j]*spbt[i][j]
            sp_distcount = sp_distcount + dem[i][j]*spbd[i][j]
            rt_timecount = rt_timecount + dem[i][j]*rtbt[i][j]
            rt_distcount = rt_distcount + dem[i][j]*rtbd[i][j]
            print(i,j,dem[i][j],spbd[i][j],rtbd[i][j],spbt[i][j],rtbt[i][j],file=outfile)
    #print("SP Base Row",i,'{:.6} {:.6} {:.6}'.format(tripcount,distcount,timecount),file=outfile)
    spb_cumtripcount = spb_cumtripcount + tripcount;
    spb_cumtime = spb_cumtime + sp_timecount;
    spb_cumdist = spb_cumdist + sp_distcount;
    rtb_cumtime = rtb_cumtime + rt_timecount;
    rtb_cumdist = rtb_cumdist + rt_distcount;
    #print("Row",i,tripcount,timecount,distcount)
#print("Shortest path base totals",'{:.8} {:.8} {:.8}'.format(cumtripcount,cumdist,cumtime),file=outfile)
#print("Shortest path base totals",'{:.8} {:.8} {:.8}'.format(spb_cumtripcount,spb_cumdist,spb_cumtime))
print(spb_cumtripcount,spb_cumdist,rtb_cumdist,spb_cumtime/60,rtb_cumtime/60)
outfile.close()

32041.0 735395949.0 735421896.7134475 18659217.216666665 18687286.305346258


## Alternative calculations using numpy array

In [14]:
sp_pht = np.array(dem)*np.array(spbt)/60
sp_pmt = np.array(dem)*np.array(spbd)
print('total pht',np.sum(sp_pht),'  average per trip',np.sum(sp_pht)/np.sum(dem))
print('total pmt',np.sum(sp_pmt),'  average per trip',np.sum(sp_pmt)/np.sum(dem))

total pht 18659217.216666665   average per trip 582.3543964503813
total pmt 735395949.0   average per trip 22951.71651945944


In [15]:
rt_pht = np.array(dem)*np.array(rtbt)/60
rt_pmt = np.array(dem)*np.array(rtbd)
print('total pht',np.sum(rt_pht),'  average per trip',np.sum(rt_pht)/np.sum(dem))
print('total pmt',np.sum(rt_pmt),'  average per trip',np.sum(rt_pmt)/np.sum(dem))

total pht 18687286.30534626   average per trip 583.2304330497257
total pmt 735421896.7134477   average per trip 22952.52634791198


## Close the files

In [18]:
f.close()
spbf.close()
rtbf.close()
outfile.close()